In [19]:
import pandas as pd
import re
import nltk
import os
import pickle
import json

In [ ]:
# Problem: LAGT not defined, just used directly

LAGT[LAGT["lemmatized_sentences"].isnull()].sample(10)

In [ ]:
# there is an invalid document:
row_index = LAGT[LAGT["doc_id"] == "tlg0530.tlg006"].index[0]
LAGT = LAGT.drop(row_index)

In [ ]:
def get_row_data(row):
    doc_id = row["doc_id"]
    # source = row["source"]
    # lemmata_source = row["lemmata_source"]
    try:
        file_path = target_path + doc_id + ".pickle"
        with open(file_path, "rb") as f:
            sentences_data = pickle.load(f)
        sentences = [sent[2] for sent in sentences_data]
        lemmatized_sentences = [[t[1] for t in sent[3] if t[2] in ["n", "a", "v", "NOUN", "PROPN", "ADJ", "VERB"]] for sent in sentences_data]
        #source = "glaux1"
        #lemmata_source = "glaux1"
    except:
        sentences = None
        lemmatized_sentences = None
    return sentences, lemmatized_sentences # , source, lemmata_source

In [ ]:
result = exprecce.apply(lambda row: pd.Series(get_row_data(row)), axis=1)

In [ ]:
result.sample(10)

In [ ]:
LAGT["sentences"] = result[0]  # Extract sentences
LAGT["lemmatized_sentences"] = result[1]

In [ ]:
LAGT.loc[1963]

In [ ]:
LAGT["lemmatized_sentences"].isnull().sum()

In [ ]:
len(LAGT)

In [ ]:
LAGT = LAGT.drop_duplicates(subset="doc_id", keep="first")
len(LAGT)

### Checking what is in what shape

In [ ]:
LAGT[LAGT["source"]=="exprecce"]

In [ ]:
def get_wordcout(doc_id):
    try:
        file_path = target_path + doc_id + ".pickle"
        with open(file_path, "rb") as f:
            sentences_data = pickle.load(f)
        wordcount = sum([len(sent_data[3]) for sent_data in sentences_data])
    except:
        wordcount = 0
    return wordcount

In [ ]:
LAGT["wordcount"] = LAGT["doc_id"].apply(get_wordcout)

In [ ]:
LAGT["wordcount"].sum()

In [ ]:
lemmatized_sentences = [s for work in LAGT["lemmatized_sentences"] for s in work]
lemmatized_sentences[:10]

In [ ]:
# update lemmatacount
LAGT["lemmatacount"] = LAGT["lemmatized_sentences"].apply(lambda x: len([l for s in x for l in s]))

In [ ]:
LAGT["lemmatacount"]

In [ ]:
LAGT['lemmata_source'] = LAGT['lemmata_source'].fillna("grecy")

In [ ]:
placeholder = "glaux_tmp"
LAGT['lemmata_source'] = LAGT['lemmata_source'].replace("glaux", placeholder)
LAGT['lemmata_source'] = LAGT['lemmata_source'].replace(placeholder, "glaux1")

In [ ]:
LAGT.groupby("lemmata_source").size() #.isnull().sum()

In [ ]:
LAGT.groupby("source").size() #.isnull().sum()


In [ ]:
LAGT.columns

In [ ]:
LAGT = LAGT[['author_id', 'doc_id', 'filename', 'author', 'title',  'sentences', 'lemmatized_sentences', 'source', 'lemmata_source', 'not_before', 'not_after', 'tlg_epithet', 'genre', 'provenience', 'wordcount', 'lemmatacount', ]]

In [ ]:
LAGT["title"].fillna("", inplace=True)
LAGT["author"].fillna("", inplace=True)


In [ ]:
LAGT[LAGT["author"].str.contains("Septuagint")]

In [ ]:
LAGT[LAGT["lemmata_source"]=="grecy"].sample(10)

In [ ]:
LAGT.to_parquet("../data/large_files/LAGT_grecy.parquet")

In [ ]:
# save metadata for future usage
LAGT[['author_id', 'doc_id', 'filename', 'author', 'title', 'source', 'lemmata_source', 'not_before',
       'not_after', 'tlg_epithet', 'genre', 'provenience', 'wordcount',
       'lemmatacount']].to_csv("../data/LAGT_v4-0_metadata.csv", index=False)

### From sents_data pickles to jsons

In [ ]:
source_path = "../data/large_files/sents_data/"
target_path = "../data/large_files/sents_data_jsons/"
try:
    os.mkdir(target_path)
except:
    pass

In [ ]:
fn = "tlg0527.tlg048.pickle"
with open(source_path + fn, "rb") as f:
    sents_data = pickle.load(f)
sents_data[:10]

In [ ]:
reformat_tags_dict = {
    "NOUN": "n",
    "VERB": "v",
    "ADJ": "a",
    "ADV": "r",
    "PRON": "p",
    "DET": "l",
    "ADP": "r",
    "CCONJ": "c",
    "SCONJ": "c",
    "PROPN": "n",
    "PUNCT": "u",
    "n" : "n",
    "v": "v",
    "a": "a",
    "r": "r",
    "p": "p",
    "l": "l",
    "c": "c",
    "u": "u" # Assuming punctuation remains "u"
}

def reformat_tags(tag):
    try:
        return reformat_tags_dict[tag]
    except:
        return "x"
    
for fn in os.listdir(source_path):
    doc_id =  fn.rpartition(".")[0]
    with open(source_path + fn, "rb") as f:
        sents_data = pickle.load(f)
    sents_data_updated = []
    for id, sent_n, sent_text, sent_data in sents_data:
        sent_data_updated = [(t[0], t[1], reformat_tags(t[2]), t[3]) for t in sent_data]
        sents_data_updated.append((doc_id, sent_n, sent_text, sent_data_updated))
    with open(target_path + doc_id + ".json", "w") as f:
        json.dump(sents_data_updated, f)

In [ ]:

fn = "tlg2640.tlg001.json"
with open(target_path + fn, "rb") as f:
    sents_data = json.load(f)
sents_data[:10]

In [ ]:
LAGT = pd.read_parquet("../data/large_files/LAGT_v4-0.parquet")
LAGT.head()


In [ ]:
path = "../data/large_files/sents_data/"
LAGT[LAGT["doc_id"].apply(lambda x: f"{x}.pickle" not in os.listdir(path))]

In [ ]:
len(set(LAGT["doc_id"]))

### Backup of an old approach...

In [ ]:
# put the string cleaning, doc creation and lemmata together into one function
def from_string_to_lemsents(string):
    try:    
        doc = get_doc(clean_string(string), segment_len=50000)
        lemmatized_sentences = get_lemmatized_sentences(doc)
    except:
        lemmatized_sentences = None
    return lemmatized_sentences

In [ ]:
grecy_lemmatized_list = []
def get_grecy_lemmata(string, lemmatized_sentences, doc_id):
    if lemmatized_sentences == None:
        lemmatized_sentences = from_string_to_lemsents(string)
        grecy_lemmatized_list.append(doc_id)
        print(len(grecy_lemmatized_list), doc_id, len(string))
    return lemmatized_sentences

#sample_lemmatized_sentences = LAGT.sample(10, random_state=1).apply(lambda row: get_grecy_lemmata(row["string"], row["lemmatized_sentences"], row["doc_id"]), axis=1)

In [ ]:
#sample_lemmatized_sentences

In [ ]:
#LAGT.loc[sample_lemmatized_sentences.index]

In [ ]:
#grecy_lemmatized_list

# Applying the main funtion

In [ ]:
%%time
### will be time consuming...
LAGT["lemmatized_sentences"] = LAGT.apply(lambda row: get_grecy_lemmata(row["string"], row["lemmatized_sentences"], row["doc_id"]), axis=1)

In [ ]:
LAGT[LAGT["lemmatized_sentences"].isnull()]

In [ ]:
missing_i = LAGT[LAGT["lemmatized_sentences"].isnull()].index

In [ ]:
string = LAGT[LAGT["lemmatized_sentences"].isnull()]["string"].tolist()[1]

In [ ]:
type(string)

In [ ]:
cleaned_string = clean_string(string)

In [ ]:
cleaned_string = re.sub("\.{3}\.*", "...", cleaned_string)

In [ ]:
doc = nlp(cleaned_string)

In [ ]:
def dealing_with_missing(string):
    string = str(string)
    cleaned_string = clean_string(string)
    cleaned_string = re.sub("\.{3}\.*", "...", cleaned_string)
    doc = nlp(cleaned_string)
    lemmatized_sentences = get_lemmatized_sentences(doc)
    return lemmatized_sentences

lemmatized_missing = LAGT[LAGT["lemmatized_sentences"].isnull()]["string"].apply(dealing_with_missing)

In [ ]:
lemmatized_missing

In [ ]:
LAGT.loc[missing_i, "lemmatized_sentences"] = lemmatized_missing

In [ ]:
LAGT.sample(10, random_state=1)

In [ ]:
LAGT["lemmata_source"] = LAGT["lemmata_source"].apply(lambda x: "grecy" if x is None else x)

# Simple explorations of what is in the lemmata

In [ ]:
lemmata_series = LAGT["lemmatized_sentences"].apply(lambda x: [l for s in x for l in s])
lemmata_all = [l for lemmata in lemmata_series for l in lemmata]
nltk.FreqDist(lemmata_all).most_common()[:100]

In [ ]:
lemmata_series = LAGT[LAGT["lemmata_source"]=="grecy"]["lemmatized_sentences"].apply(lambda x: [l for s in x for l in s])
lemmata_all = [l for lemmata in lemmata_series for l in lemmata]
nltk.FreqDist(lemmata_all).most_common()[:100]

In [ ]:
LAGT.to_json("../data/large_files/LAGT_grecy_20240116.json")

In [ ]:
#s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")
s.write_file("SDAM_data/AGT/LAGT_grecy_20240116.json", LAGT)